In [1]:
# Install the required package
# %pip install google-generativeai
# %pip install -qU "google-genai>=1.0.0"

# import google as genai
# from flask import Flask, request, jsonify
# import os
# from dotenv import load_dotenv
# load_dotenv()


In [2]:
from google import genai
from flask import Flask, request, jsonify
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

### Define the API
To emulate a resturant's ordering system, define functions for managing the customer's order: adding, editing, clearing, confirming and fulfilling.

These functions track the customer's order using the global variables order (the in-progress order) and placed_order (the confirmed order sent to the kitchen). Each of the order-editing functions updates the order, and once placed, order is copied to placed_order and cleared.

In [4]:
from typing import Optional
from random import randint

order = []
placed_order = []

def add_to_order(food: str, quantity: int, modifiers: Optional[list[str]] = None) -> None:
    """Add food to the order."""
    if modifiers is None:
        modifiers = []
    order.append({"food": food, "quantity": quantity, "modifiers": modifiers})
    # return f"Added {quantity} of {food} to your order."

def remove_from_order(n: int) -> str:
    """Remove food from the order."""
    if 1 <= n <= len(order):
        item= order.pop(n - 1)
        return f"Removed {item['quantity']} of {item['food']} from your order."
    else:
        return "Invalid item"

def get_order() -> list[dict]:
    """Get the current order."""
    return order

def clear_order() -> None:
    """Clear the order."""
    order.clear()


def confirm_order() -> bool:
    """Asks the customer if the order is correct.
    
    Returns:
        The user's confirmation."""
    print(f"Your order:")
    if not order:
        print("No items in the order.")
        return False
    
    # for quantity, food, modifiers in order.values:
    #     print(f"{quantity} of {food}")
    #     if modifiers:
    #         print(f'   - {", ".join(modifiers)}')

    for i, item in enumerate(order, start=1):
        food = item["food"]
        quantity = item["quantity"]
        modifiers = ", ".join(item["modifiers"]) if item["modifiers"] else "No modifiers"
        print(f" {quantity} x {food} ({modifiers})")

    return input('Is this correct? (yes/no): ')
        # return response == "yes"
    
def place_order() -> Optional[int]:
    """Place the order only if confirmed
    
    Returns:
        Estimated preparation time if confirmed, None otherwise"""
    if confirm_order():
        placed_order[:] = order.copy()
        clear_order()
        return randint(1, 10)
    else:
        print("Order not placed.")
        return None
    



    # return randint(1, 10)


In [5]:
# Test it out 
clear_order()
add_to_order('Pizza', 3)
add_to_order("Vegetables", 2)
# remove_from_order(1)
remove_from_order(1)
confirm_order()




Your order:
 2 x Vegetables (No modifiers)


'yes'

### Define Prompt

In [6]:
BISTRO_BOT_PROMPT = """\You are a food order taking system and you are restricted to talk only about foods on the MENU. Do not talk about anything but ordering MENU foods for the customer, ever.
Your goal is to do place_order after understanding the menu items and any modifiers the customer wants.
Add items to the customer's order with add_to_order, remove specific items with remove_item, and reset the order with clear_order.
To see the contents of the order so far, call get_order (by default this is shown to you, not the user)
Always confirm_order with the user (double-check) before calling place_order. Calling confirm_order will display the order items to the user and returns their response to seeing the list. Their response may contain modifications.
Always verify and respond with food and modifier names from the MENU before adding them to the order.
If you are unsure a food or modifier matches those on the MENU, ask a question to clarify or redirect.
You only have the modifiers listed on the menu below: Rice options, Spaghetti, Chicken, Fries, special requests.
Once the customer has finished ordering items, confirm_order and then place_order.

Hours: Tues, Wed, Thurs, 10am to 2pm
Prices: All drinks are free.

MENU:
Rice options:
Jollof Rice
Native Rice
Veggies Stir Fry
Thai Jolly Shrimp Rice
Suya Fried Rice


Spaghetti:
Stir Fry Fusilli
Stir Fry Spaghetti
Native-Style Spaghetti
Misky Sauce Penny
Misky Sauce Fusilli
Smokey Jollof Penny

Protein:
Turkey Wings
Chicken Wings (6 pieces)
Chicken Wings (15 pieces)
Sauteed Chicken
Sauteed Prawns
Bioled Eggs
Sauteed Veggies

Fries:
Sweet Potato Fries
Yam Fries
French Fries


"""

### Set Up Model

In [7]:
# %pip install google-api-core

In [8]:
# Install the required package
# %pip install google-api-core

from google.genai import types
from google.api_core import retry

ordering_system = [
    add_to_order,
    get_order, 
    remove_from_order,
    clear_order,
    confirm_order,
    place_order,
]
model_name = "gemini-2.0-flash"

chat = client.chats.create(
    model= model_name,
    config=types.GenerateContentConfig(
        tools=ordering_system,
        system_instruction=BISTRO_BOT_PROMPT,
    ),
)
placed_order=[]
order=[]

In [9]:
from IPython.display import display, Markdown

print("Welcome to the Bistro Bot! How can I help you today?")
while not placed_order:
    response = chat.send_message(input("> "))
    display(Markdown(response.text))

print("\n\n")
print("[bistro bot] Your order has been placed!")
print()
print("Your order:")
print(f"  {placed_order}\n")
print("Thank you for ordering with us!")

Welcome to the Bistro Bot! How can I help you today?


Okay, I have added one order of Native Rice to your order.


Ok, I have added one Yam Fries to your order. Anything else?


I am sorry, I do not have yam porridge. Would you like to order anything else from the menu?


Your order:
 1 x Native Rice (No modifiers)
 1 x Yam Fries (No modifiers)
Your order:
 1 x Native Rice (No modifiers)
 1 x Yam Fries (No modifiers)


Okay, your order has been placed and will be ready in 5 minutes.





[bistro bot] Your order has been placed!

Your order:
  [{'food': 'Native Rice', 'quantity': 1, 'modifiers': []}, {'food': 'Yam Fries', 'quantity': 1, 'modifiers': []}]

Thank you for ordering with us!
